In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib

# Load data from CSV file
def load_data(csv_file_path):
    # Read the CSV file
    df = pd.read_csv(csv_file_path)

    # Convert columns to numeric (in case they're loaded as strings)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Display data information
    print("Data overview:")
    print(df.head())
    print("\nClass distribution:")
    print(df['Human Status'].value_counts())

    return df

def prepare_data(df):
    # Extract features and target
    X = df[['Receiver 1', 'Receiver 2', 'Receiver 3']].values
    y = df['Human Status'].values

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Save the scaler
    joblib.dump(scaler, 'scaler_c.save')  

    X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

    # One-hot encode targets
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    return X_train_cnn, X_test_cnn, y_train_categorical, y_test_categorical, y_train, y_test

# Build CNN model
def create_cnn_model(num_classes):
    model = Sequential()

    # CNN layers
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(3, 1)))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Train model with early stopping
def train_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=32):
    # Define early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
        verbose=1
    )

    return model, history

# Evaluate model and visualize results
def evaluate_model(model, X_test, y_test_categorical, y_test, history):
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
    print(f"Test accuracy: {test_accuracy:.4f}")

    # Get predictions
    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred,
                              target_names=['Empty (0)', 'Stationary (1)', 'Moving (2)']))

    # Confusion matrix
    plt.figure(figsize=(10, 7))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Empty', 'Stationary', 'Moving'],
                yticklabels=['Empty', 'Stationary', 'Moving'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix_c.png')
    plt.close()

    # Training history
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.savefig('training_history_c.png')
    plt.close()

    return y_pred

# Function to make predictions on new data
def predict_human_status(model, new_data, scaler):
    # Scale new data
    new_data_scaled = scaler.transform(new_data)

    # Reshape for CNN
    new_data_cnn = new_data_scaled.reshape(new_data_scaled.shape[0], new_data_scaled.shape[1], 1)

    # Make predictions
    predictions = model.predict(new_data_cnn)
    predicted_classes = np.argmax(predictions, axis=1)

    # Map predictions to labels
    status_map = {0: 'Empty Room', 1: 'Stationary Person', 2: 'Moving Person'}
    predicted_labels = [status_map[cls] for cls in predicted_classes]

    return predicted_classes, predicted_labels, predictions

# Main execution function
def main(csv_file_path):
    # Load and prepare data
    df = load_data(csv_file_path)
    X_train, X_test, y_train_cat, y_test_cat, y_train, y_test = prepare_data(df)

    # Get number of classes
    num_classes = len(np.unique(y_train))
    print(f"Number of classes: {num_classes}")

    # Create and train model
    model = create_cnn_model(num_classes)
    model.summary()

    trained_model, history = train_model(model, X_train, y_train_cat, X_test, y_test_cat)

    # Evaluate model
    y_pred = evaluate_model(trained_model, X_test, y_test_cat, y_test, history)

    # Save the model
    trained_model.save('rssi_human_detection_model_c.h5')
    print("Model saved as 'rssi_human_detection_model.h5'")

    return trained_model

if __name__ == "__main__":
    # Specify the path to your CSV file
    csv_file_path = "D:\\WiSee\\data\\finalDataset.csv"

    # Run the main function
    model = main(csv_file_path)

Data overview:
   Tile No  Receiver 1  Receiver 2  Receiver 3  Human Status
0        1         -37         -42         -34             0
1        1         -37         -42         -32             0
2        1         -37         -42         -32             0
3        1         -36         -41         -34             0
4        1         -39         -42         -35             0

Class distribution:
Human Status
2    818
1    734
0    629
Name: count, dtype: int64
Number of classes: 3


d:\WiSee\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 2, 64)          │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 2, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,627 (25.89 KB)

 Trainable params: 6,627 (25.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6743 - loss: 0.9106 - val_accuracy: 0.9062 - val_loss: 0.3662
Epoch 2/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8766 - loss: 0.3584 - val_accuracy: 0.8993 - val_loss: 0.2437
Epoch 3/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9025 - loss: 0.2570 - val_accuracy: 0.9062 - val_loss: 0.2207
Epoch 4/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9072 - loss: 0.2313 - val_accuracy: 0.9199 - val_loss: 0.2037
Epoch 5/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9215 - loss: 0.2204 - val_accuracy: 0.9245 - val_loss: 0.1868
Epoch 6/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9126 - loss: 0.2030 - val_accuracy: 0.9336 - val_loss: 0.1848
Epoch 7/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9214 - loss: 0.1978 - val_accuracy: 0.9382 - val_loss: 0.1605
Epoch 8/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9246 - loss: 0.1584 - val_accuracy: 0.9382

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Classification Report:
                precision    recall  f1-score   support

     Empty (0)       0.99      0.99      0.99       126
Stationary (1)       0.97      0.98      0.97       147
    Moving (2)       0.99      0.98      0.98       164

      accuracy                           0.98       437
     macro avg       0.98      0.98      0.98       437
  weighted avg       0.98      0.98      0.98       437



Model saved as 'rssi_human_detection_model.h5'
